In [1]:
import os
import sys
import pandas as pd
import numpy as np
import warnings

from scipy.stats import betabinom

from sklearn.utils import resample
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error


import gc

root_path = root_path = os.path.realpath('../..')
try:
    import causaltune
except ModuleNotFoundError:
    sys.path.append(os.path.join(root_path, "auto-causality"))

from causaltune import CausalTune
from causaltune.data_utils import CausalityDataset

from flaml import AutoML
import wise_pizza as wp

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

warnings.filterwarnings("ignore")

%load_ext autoreload
%autoreload 2

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


## Propensity Score Weighting in CausalTune 

CausalTune effect estimation consists of multiple models that can / need to be fitted.

1. Propensity model to estimate treatment propensities from features $\mathbb{E}[T|X,W]$.
2. Outcome model to estimate outcomes from features $\mathbb{E}[Y|X,W]$
3. The final causal inference estimator which requires additional hyperparamter tuning.

In this notebook, we focu on the Propensity Model (1.).

There are four options to finding a propensity model.

1. **[Default:] use a dummy estimator.**
   - natural option for a computationally easy model / when perfect randomisation of the treatment is given 
2. **Letting AutoML fit the propoensity model,**
   - has all the advantages of using an elaborate propensity weighting model  
3. **supply a custom sklearn-compatible prediction model,**
   - for more flexibility in terms of propensity prediction model
4. **supply an array of custom propensities to treat.** 
   - can be used, e.g. with custom propensities to treat based on an optimisation procedure such as Thompson sampling when there is an expected benefit from treating some subjects with higher propensity than others

### Data Generating Process

The DGP in this example can be described as follows:

\begin{align}
    & T \sim Bernoulli(.5)\\
    %T &= h(\beta \cdot W) + \eta \\ 
    Y &= T* \rho + m(\gamma \cdot X) + \epsilon \\
    &\rho = 0.01 \\
    %& W \sim BetaBinom(8, 600, 400) \\ % later give individual probabilities
    %\beta & = (.1, .2, .3, .4, .5, 0 , 0, ...)\\
    & X \sim hypergeometric(5, 5, 8)\\
    & \gamma \sim Uniform([.5, 1.5]) \\
    & m(x) = .5*x

\end{align}
In particular, we assume 
- perfect randomisation of the treatment as we are replicating an AB test environment and
- a constant treatment effect (for now).

In [25]:

def generate_synth_data_with_categories(
    n_samples=10000,
    n_x=10,
) -> CausalityDataset:
    n_w = 3
    T = np.random.binomial(1, 0.5, size=(n_samples,))
    X = np.random.hypergeometric(5, 5, 8, size=(n_samples, n_x))
    W = betabinom.rvs(8, 600, 400, size=(n_samples, n_w))
    epsilon = 3*np.random.uniform(low=-1, high=1, size=(n_samples,))
    gamma = np.random.uniform(low=0.5, high=1.5, size=(n_x,))
    rho = lambda x: 0.01
    feature_transform = lambda x: 0.5 * x

    Y = T.T * rho(X[:, : int(n_x / 2)])   + feature_transform(np.matmul(gamma.T, X.T)) + epsilon

    features = [f"X{i+1}" for i in range(n_x)]
    features_w = [f"W{i+1}" for i in range(n_w)]
    df = pd.DataFrame(np.array([*X.T, T, Y, *W.T]).T, columns=features + ["variant", "Y"] + features_w) 
    df['prop_modifiers'] = .5*np.ones(len(df))
    
    cd = CausalityDataset(
        data=df,
        treatment="variant",
        outcomes=["Y"],
        propensity_modifiers=['prop_modifiers']

    )
    return cd
cd = generate_synth_data_with_categories(n_samples=10000)
cd.preprocess_dataset()

In [ ]:
# CausalTune configuration
components_time_budget = 10
train_size = 0.7

target = cd.outcomes[0]

### 1. DEFAULT: Dummy propensity model


The dummy propensity model identifies a constant propensity to treat given by $\frac{\text{Treatment Group Size}}{\text{Total Sample Size}}  $.

In [6]:
ct = CausalTune(
    propensity_model='dummy',
    components_time_budget=components_time_budget,
    metric="energy_distance",
    train_size=train_size,
    verbose=0
)   
ct.fit(data=cd, outcome=target)

[flaml.tune.tune: 05-18 16:20:36] {493} WARNING - Using CFO for search. To use BlendSearch, run: pip install flaml[blendsearch]
[flaml.tune.tune: 05-18 16:20:36] {636} INFO - trial 1 config: {'estimator': {'estimator_name': 'backdoor.causaltune.models.NaiveDummy'}}


Fitting a Propensity-Weighted scoring estimator to be used in scoring tasks
Initial configs: [{'estimator': {'estimator_name': 'backdoor.causaltune.models.NaiveDummy'}}, {'estimator': {'estimator_name': 'backdoor.causaltune.models.Dummy'}}, {'estimator': {'estimator_name': 'backdoor.econml.metalearners.SLearner'}}, {'estimator': {'estimator_name': 'backdoor.econml.metalearners.DomainAdaptationLearner'}}, {'estimator': {'estimator_name': 'backdoor.econml.dr.ForestDRLearner', 'min_propensity': 1e-06, 'n_estimators': 100, 'min_samples_split': 5, 'min_samples_leaf': 5, 'min_weight_fraction_leaf': 0.0, 'max_features': 'auto', 'min_impurity_decrease': 0.0, 'max_samples': 0.45, 'min_balancedness_tol': 0.45, 'honest': True, 'subforest_size': 4}}, {'estimator': {'estimator_name': 'backdoor.econml.dml.CausalForestDML', 'drate': True, 'n_estimators': 100, 'criterion': 'mse', 'min_samples_split': 10, 'min_samples_leaf': 5, 'min_weight_fraction_leaf': 0.0, 'max_features': 'auto', 'min_impurity_decr

[flaml.tune.tune: 05-18 16:20:36] {198} INFO - result: {'energy_distance': 0.004698108114425281, 'estimator_name': 'backdoor.causaltune.models.NaiveDummy', 'scores': {'train': {'erupt': 17.48511742445152, 'norm_erupt': 17.46274622719086, 'qini': -12.004135590982141, 'auc': 0.36688909036680456, 'values':       variant          Y         p  policy  norm_policy   weights
0           0  15.537179  0.500429    True        False  0.000000
1           0  18.229643  0.500429    True        False  0.000000
2           0  17.227617  0.500429    True        False  0.000000
3           0  16.756012  0.500429    True        False  0.000000
4           1  18.099593  0.500429    True        False  1.998287
...       ...        ...       ...     ...          ...       ...
6995        1  15.391084  0.500429    True         True  1.998287
6996        0  19.831789  0.500429    True        False  0.000000
6997        1  19.462900  0.500429    True        False  1.998287
6998        1  17.604315  0.500429 

In [8]:
ct.propensity_model

DummyClassifier()

Difference in means estimate (naive ATE):

In [9]:
ct.scorer.naive_ate(cd.data[cd.treatment], cd.data[target])[0]

0.008980048095487803

CausaTune ATE estimate:

In [10]:
ct.effect(ct.test_df).mean()

-0.03120091211534311

### 2. Propensity model estimation via AutoML

The propensity score weighting estimation via AutoML is as simple as selecting `propensity_model='auto'`. 

The computational intensity can then be adapted via supplying a `components_budget_time`.


In [ ]:
ct = CausalTune(
    propensity_model='auto',
    components_time_budget=components_time_budget,
    metric="energy_distance",
    train_size=train_size,
    verbose=0
)   
ct.fit(data=cd, outcome=target)

### 3. Propensity model estimation with a custom model

A custom propensity model that has an sklearn-style `fit` and `predict_proba` method can be supplied as a propensity model.

In [32]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

params_propensity = { 
    'n_estimators': [200, 500],
    'max_depth' : [None, 3,6],
    'criterion' :['gini', 'entropy']
}

propensity_model = GridSearchCV(
    RandomForestClassifier(),
    params_propensity
    )

ct = CausalTune(
    propensity_model=propensity_model,
    components_time_budget=components_time_budget,
    metric="energy_distance",
    train_size=train_size,
)   
ct.fit(data=cd, outcome=target)

Fitting a Propensity-Weighted scoring estimator to be used in scoring tasks


[flaml.tune.tune: 05-18 18:08:49] {493} WARNING - Using CFO for search. To use BlendSearch, run: pip install flaml[blendsearch]
[flaml.tune.tune: 05-18 18:08:49] {636} INFO - trial 1 config: {'estimator': {'estimator_name': 'backdoor.causaltune.models.NaiveDummy'}}


Initial configs: [{'estimator': {'estimator_name': 'backdoor.causaltune.models.NaiveDummy'}}, {'estimator': {'estimator_name': 'backdoor.causaltune.models.Dummy'}}, {'estimator': {'estimator_name': 'backdoor.econml.metalearners.SLearner'}}, {'estimator': {'estimator_name': 'backdoor.econml.metalearners.DomainAdaptationLearner'}}, {'estimator': {'estimator_name': 'backdoor.econml.dr.ForestDRLearner', 'min_propensity': 1e-06, 'n_estimators': 100, 'min_samples_split': 5, 'min_samples_leaf': 5, 'min_weight_fraction_leaf': 0.0, 'max_features': 'auto', 'min_impurity_decrease': 0.0, 'max_samples': 0.45, 'min_balancedness_tol': 0.45, 'honest': True, 'subforest_size': 4}}, {'estimator': {'estimator_name': 'backdoor.econml.dml.CausalForestDML', 'drate': True, 'n_estimators': 100, 'criterion': 'mse', 'min_samples_split': 10, 'min_samples_leaf': 5, 'min_weight_fraction_leaf': 0.0, 'max_features': 'auto', 'min_impurity_decrease': 0.0, 'max_samples': 0.45, 'min_balancedness_tol': 0.45, 'honest': Tru

[flaml.tune.tune: 05-18 18:08:50] {198} INFO - result: {'energy_distance': 0.006390931997504623, 'estimator_name': 'backdoor.causaltune.models.NaiveDummy', 'scores': {'train': {'erupt': 18.479582727951996, 'norm_erupt': 18.503352928137463, 'qini': -81.67256403906752, 'auc': -13.968875050095626, 'values':       variant          Y      p  policy  norm_policy   weights
0           0  20.034063  0.140   False         True  1.885650
1           1  19.323346  0.780   False         True  0.000000
2           1  21.762711  0.780   False         True  0.000000
3           0  15.911813  0.190   False         True  2.002048
4           0  18.271576  0.120   False         True  1.842794
...       ...        ...    ...     ...          ...       ...
6995        0  21.586115  0.090   False         True  1.782043
6996        0  19.783017  0.210   False         True  2.052733
6997        1  16.894379  0.790   False         True  0.000000
6998        0  21.003258  0.175   False         True  1.965647
6

KeyboardInterrupt: 

### 4. Supplying individual treatment propensities

In some settings, the experiment / study is based on heterogeneous treatment propensities known to the researcher / experimenter. An array of treatment propensities can be directly supplied to CausalTune in the data instantiation of the `CausalityDataset`. This can, e.g. be done by 
```
cd = CausalityDataset(
    ...
    propensity_modifiers=[<individual_treatment_propensity_column_name>]
    ...
)
```
and then using the `passthrough_model` as follows

In [30]:
cd = generate_synth_data_with_categories(n_samples=10000)
cd.preprocess_dataset()
cd.propensity_modifiers


['prop_modifiers']

In [ ]:
from causaltune.models.passthrough import passthrough_model


cd = generate_synth_data_with_categories(n_samples=10000)

cd.preprocess_dataset()
print(cd.data.head())
print(cd.propensity_modifiers)

propensity_model=passthrough_model(
    cd.propensity_modifiers, include_control=False
    )

ct = CausalTune(
    propensity_model=propensity_model,
    components_time_budget=components_time_budget,
    metric="energy_distance",
    train_size=train_size,
    verbose=0
)   
ct.fit(data=cd, outcome=target)